# Finetuning using Pytorch
(using model is pretrained)<br>

1. finetuning : update all model's parameter for new task = retrain whole model
2. feature extraction : only update the final layer weights

모델마다 구조가 다르기 때문에 일반화된 finetuning 함수를 짜기는 어려움<br>
하지만 일반화된 규칙은 있기 때문에 그때그때 custom하게 수정하면 됨<br>
(참고 : https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)

In [1]:
import numpy as np
import os
import sys

import torch
from torch import nn
from torch.nn import functional as Flatten
from torch import optim
import torchvision
from torchvision import datasets, models, transforms

from source import functions
from source import cifar_dataloader


print('python version : ', sys.version)
print('torch version : ', torch.__version__)
print('torchvision version : ', torchvision.__version__)
print('device number : ', torch.cuda.device_count())

python version :  3.8.13 (default, Oct 21 2022, 23:50:54) 
[GCC 11.2.0]
torch version :  1.13.0+cu117
torchvision version :  0.14.0+cu117
device number :  4


In [85]:
from source import functions
import importlib
importlib.reload(functions)

<module 'source.functions' from '/home/choiyj/pytorch_study/source/functions.py'>

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

## new data and model

In [2]:
model_names = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_classes = 10
batch_size = 8
num_epochs = 15

In [5]:
# fine tuning의 핵심
# source.functions에 포함되어있음

# def set_parameter_requires_grad(model, feature_extracting):
#     if feature_extracting:
#         for param in model.parameters():
#             param.requires_grad = False

In [ ]:
# Initialize the model for this run
model_name = 'resnet'
models, input_size = functions.get_model(model_name, pretrained=True, transfer=True, feature_extract=True, num_classes=num_classes)

# Print the model we just instantiated
print(models.fc)

In [10]:
path = './../data/'
dataloaders_dict = cifar_dataloader.get_cifar10_dataloader(path, input_size, 16, 4)

Files already downloaded and verified
Files already downloaded and verified


## optimizer
optimizer에는 model.parameters()를 넣어주되 학습할 parameter와 학습하지 않을 parameter를 구분하여 넣어준다.

In [12]:
models = models.to(device)

# optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
params_to_update = functions.get_model_parameters(models, feature_extract=True)
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [67]:
# optimizer에 넣는 parameter는 다음과 같이 list로 넣어주면 된다.
# params_to_update[0], params_to_update[1]

## training

In [13]:
criterion = nn.CrossEntropyLoss()

# Train and evaluate
# inception은 loss가 두가지이기 때문에 다르게 학습되므로 별도의 분기점 존재
model_ft, hist = functions.train_model(
    model = models, 
    dataloaders = dataloaders_dict, 
    criterion = criterion, 
    optimizer = optimizer, 
    device=device, 
    num_epochs=num_epochs, 
    is_inception=(model_name=="inception")
    )

Epoch 0/14
----------
train Loss: 1.4689 Acc: 0.4905
val Loss: 0.8253 Acc: 0.7194

Epoch 1/14
----------
train Loss: 1.3846 Acc: 0.5244
val Loss: 0.8455 Acc: 0.7178

Epoch 2/14
----------
train Loss: 1.3795 Acc: 0.5297
val Loss: 0.7397 Acc: 0.7463

Epoch 3/14
----------
train Loss: 1.3740 Acc: 0.5313
val Loss: 0.7972 Acc: 0.7329

Epoch 4/14
----------
train Loss: 1.3715 Acc: 0.5333
val Loss: 0.7930 Acc: 0.7293

Epoch 5/14
----------
train Loss: 1.3755 Acc: 0.5322
val Loss: 0.7769 Acc: 0.7418

Epoch 6/14
----------
train Loss: 1.3782 Acc: 0.5279
val Loss: 0.7846 Acc: 0.7354

Epoch 7/14
----------
train Loss: 1.3660 Acc: 0.5348
val Loss: 0.7843 Acc: 0.7343

Epoch 8/14
----------
train Loss: 1.3719 Acc: 0.5328
val Loss: 0.7700 Acc: 0.7354

Epoch 9/14
----------
train Loss: 1.3741 Acc: 0.5296
val Loss: 0.7501 Acc: 0.7409

Epoch 10/14
----------
train Loss: 1.3695 Acc: 0.5337
val Loss: 0.7707 Acc: 0.7380

Epoch 11/14
----------
train Loss: 1.3680 Acc: 0.5329
val Loss: 0.7890 Acc: 0.7284

Ep

# scatch 모델과의 비교
- input_size만 신경쓰면 그대로 사용 가능

In [ ]:

scratch_model,_ = functions.get_model(model_name, pretrained=False, transfer=True, feature_extract=False, num_classes=num_classes)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = functions.train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, device, num_epochs=num_epochs, is_inception=(model_name=="inception"))